# Data Generator 

En este archivo estara la script encargada de generar informacion y de meterla en SQL de forma sencilla


In [1]:
import pymysql
from random_username.generate import generate_username
from random import randint
from datetime import date
from faker import Faker

In [2]:
# Conexion

conn = pymysql.connect(host='localhost', port=3306, user='root', passwd='', db='deuzumdb')
cut = conn.cursor()

# Faker
fake = Faker()


In [3]:
preguntasSeg = [1,4,6,7,8]

En la cell de abajo se encuentran todas las funciones cuya funcionalidad es generica sobre las bases de datos


In [4]:
def get_random_element_from_table(table, column):
    get_user_SQL = "SELECT `"+column+"` FROM `"+table+"` ORDER BY RAND() LIMIT 1"
    cut.execute(get_user_SQL)
    return cut.fetchone()

def get_random_elements_from_table(table,column,c_size):
    get_user_SQL = "SELECT `"+column+"` FROM `"+table+"` ORDER BY RAND() LIMIT 2"
    cut.execute(get_user_SQL)
    return cut.fetchmany(c_size)






En la cell de abajo se encuentran las funciones con las que podemos crear la información que queremos añadir a la base de datos. Las funciones que hemos creado para failitar este apartado son:

- *crear_info_usuarios_basica*: Crea cuentas de usuario aleatorias 
- *crear_info_cuenta_basica*: Crea cuentas de usuarios aleatorios
- *crear_info_historial_basica*: Crea transacciones aleatorias entre usuarios


El listado de parametros es el siguiente:




| Parametro                | Descripción                                      |
|--------------------------|--------------------------------------------------|
| c_size                   | Cantidad de elementos que el programa va a crear |
| min_[nombre de variable] | Valor mínimo que se le asignará a la variable    |
| max_[nombre de variable] | Valor máximo que se le asignará a la variable    |


In [26]:
def crear_info_usuarios_basica(c_size):
    add_SQL = "INSERT INTO `usuario` (`usuario`, `contraseña`, `preg_seguridad`, `resp_seguridad`, `permisos` ) VALUES (%s, %s, %s, %s, %s)"
    for i in range(0, c_size):
        #user
        tV_user, tV_pass, tV_res  = generate_username(3)
        cut.execute(add_SQL,(tV_user+str(randint(0,10000)),tV_pass,str(preguntasSeg[randint(0,4)]),tV_res,"3"))
        conn.commit()
        
def crear_info_cuentas_basica(c_size, min_num = 1, max_num = 1024, min_din = 1, max_din = 1024):
    add_SQL = "INSERT INTO `cuenta` (`id_usuario`, `dinero`, `tipo_cuenta`) VALUES (%s, %s, %s)"
    for i in range(0, c_size):
        #user
        tV_user = get_random_element_from_table("usuario", "id")
        tV_num, tV_din = randint(min_num,max_num), randint(min_din,max_din)
        
        cut.execute(add_SQL,(tV_user,tV_din,"gastos"))
        conn.commit()

def crear_transaccion(c_size, min_din = 1, max_din = 1024):
    add_SQL = "INSERT INTO `transaccion` (`source`, `destino`, `dinero`) VALUES (%s,%s,%s)"
    for i in range(0, c_size):
        #user
        tV_emi, tV_rece = get_random_elements_from_table("cuenta","numero_cuenta",2)
        tV_din = randint(min_din,max_din)
        
        cut.execute(add_SQL,(tV_emi, tV_rece,tV_din))
        conn.commit()
        
        
def crear_info_info_ususario(id):
    add_SQL = "INSERT INTO `infousuario` (`id`,`nombre`, `apellido`, `telefono`, `email`,`direccion`,`fecha_nacimiento`,`sexo`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
    nombreS = fake.name()
    nombre, apellido = nombreS.split(" ")
    calle = fake.address()[1:20]
    email = fake.email()
    data = (str(id),nombre, apellido, str(randint(100000000,999999999)), email, calle, "NULL", "NULL")
    cut.execute(add_SQL,data)
    conn.commit()
    
def crear_grupo(idq):
    
    add_SQL = "INSERT INTO `grupo` (`id`,`nombre`,`permisos`) VALUES (%s,%s)"
    cut.execute(add_SQL,(str(idq), generate_username(1)[0]))
    conn.commit()
                
def crear_info_grupo():
    ida = get_random_element_from_table("usuario", "id")
    grupo = get_random_element_from_table("grupo", "nombre")
    add_SQL = "INSERT INTO `gruposmiembros` (`id_grupo`, `miembro`, `permisos`) VALUES (%s,%s, %s)"
    cut.execute(add_SQL,(grupo, ida,2*randint(0,1)+1))
    conn.commit()
                
                
def crear_info_proyectos():
                
    add_SQL = "INSERT INTO `Proyectos` (`nombre`, `nombre`,`grupo` ) VALUES (%s,%s,%s)"



In [29]:
#crear_info_usuarios_basica(1000)
#conn.close()
#crear_info_cuentas_basica(2400)
#crear_transaccion(100)
crear_info_info_ususario(3)
#crear_grupo(2)
#crear_info_grupo()

IntegrityError: (1452, 'Cannot add or update a child row: a foreign key constraint fails (`deuzumdb`.`infousuario`, CONSTRAINT `infousuario_ibfk_1` FOREIGN KEY (`id`) REFERENCES `usuario` (`id`))')

In [41]:
maxLen, maxLenp = 0,0
for j in range(0,1000):
        
    maxLen = max(maxLen, len(generate_username(1)[0]))
    if(maxLen> maxLenp):
        print(str(maxLen) + " at " + str(j))
        maxLenp = maxLen
print(str(maxLen))

11 at 0
14 at 1
15 at 6
17 at 8
17
